### **讀取資料**

In [3]:
import sys
import os

# 新的 project_root 指向 common 的上一層
project_root = os.path.abspath(os.path.join("..", "QuantCommon"))
if project_root not in sys.path:
    sys.path.append(project_root)

from utils.tools import read_file
from utils.processing import get_dollar_bars, apply_cusum_filter, getDailyVol
import numpy as np
import pandas as pd
import numpy as np


clusters = pd.read_csv('clusters.csv', index_col=0)
print(f"XAUUSD 在第{clusters.loc['XAUUSD_M1', 'cluster']}群")

XAUUSD 在第2群


In [6]:
group = clusters[clusters['cluster'] == clusters.loc['XAUUSD_M1', 'cluster']]

data = dict({})
for i in group.index:
    filepath = os.path.join(project_root, "data", "FI", "M1",f"{i}.csv")
    df = pd.read_csv(filepath, index_col="time", parse_dates=True)
    df = get_dollar_bars(df)
    data[i] = df

: 

In [2]:
vol = getDailyVol(data["close"], span0=20)
cusum_events  = apply_cusum_filter(data, volatility=vol).index

CUSUM Bars Count: 17589


In [3]:
def add_vertical_barrier(tEvents, close, num_days=2):
    t1 = close.index.searchsorted(tEvents + pd.Timedelta(days=num_days))
    t1 = t1[t1 < close.shape[0]]
    t1 = pd.Series(close.index[t1], index=tEvents[:t1.shape[0]])
    return t1
vertical_barriers = add_vertical_barrier(cusum_events, data["close"], num_days=2)
vertical_barriers.head()

time
2021-01-04 00:05:00   2021-01-06 00:07:00
2021-01-04 00:19:00   2021-01-06 00:23:00
2021-01-04 00:29:00   2021-01-06 00:33:00
2021-01-04 00:33:00   2021-01-06 00:33:00
2021-01-04 00:56:00   2021-01-06 01:01:00
Name: time, dtype: datetime64[ns]

In [4]:
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing as mp

def mpPandasObj(func, pdObj, numThreads=24, mpBatches=1, **kargs):
    """
    通用的 multiprocessing 封裝器（支援 Series/DataFrame 分批運算）
    pdObj: dict of name→array-like, 會把每個 name 切片後放進 func
    """
    def linParts(numAtoms, numPieces):
        edges = np.linspace(0, numAtoms, numPieces + 1)
        edges = np.ceil(edges).astype(int)
        return list(zip(edges[:-1], edges[1:]))

    # 切成 numThreads * mpBatches 段
    parts = linParts(len(next(iter(pdObj.values()))), numThreads * mpBatches)
    jobs = []
    for start, end in parts:
        kargs_ = kargs.copy()
        for name, arr in pdObj.items():
            kargs_[name] = arr[start:end]
        jobs.append(delayed(func)(**kargs_))
    out = Parallel(n_jobs=numThreads)(jobs)

    # 如果回傳的是 DataFrame/Series，就 concat 起來
    if isinstance(out[0], (pd.DataFrame, pd.Series)):
        return pd.concat(out, axis=0)
    return out

def applyPtSlOnT1(close, events, pt_sl, molecule):
    """
    在每個事件的 [start, t1] 區間內，找第一次觸及 pt/sl 的時間
    回傳包含欄位 ['t1','pt','sl'] 的 DataFrame
    """
    ev = events.loc[molecule]
    out = ev[['t1']].copy()
    # profit‐taking & stop‐loss 閾值
    pt = pd.Series(pt_sl[0] * ev['trgt'], index=ev.index) if pt_sl[0] > 0 else pd.Series(index=ev.index, dtype=float)
    sl = pd.Series(-pt_sl[1] * ev['trgt'], index=ev.index) if pt_sl[1] > 0 else pd.Series(index=ev.index, dtype=float)

    # 初始化
    out['pt'] = pd.NaT
    out['sl'] = pd.NaT

    for loc, t1 in ev['t1'].fillna(close.index[-1]).items():
        path = close.loc[loc:t1]
        ret  = (path / close.loc[loc] - 1) * ev.at[loc, 'side']

        # 第一次觸及 pt、sl
        hits_pt = ret[ret > pt.loc[loc]]
        hits_sl = ret[ret < sl.loc[loc]]

        out.at[loc, 'pt'] = hits_pt.index[0] if not hits_pt.empty else t1
        out.at[loc, 'sl'] = hits_sl.index[0] if not hits_sl.empty else t1

    return out

def get_events(close,
               t_events,
               pt_sl,
               target,
               min_ret=0,
               num_threads=None,
               vertical_barrier_times=None,
               side_prediction=None):
    """
    close: pd.Series 收盤價
    t_events: list or index of event start times
    pt_sl: [pt_multiplier, sl_multiplier]
    target: pd.Series, trgt 大小
    min_ret: 最小 trgt 閥值
    vertical_barrier_times: pd.Series of same index as t_events (可 None)
    side_prediction: pd.Series of same index as t_events (可 None)
    """
    if num_threads is None:
        num_threads = mp.cpu_count()

    # 1. 篩選 target
    target = target.reindex(t_events).dropna()
    target = target[target > min_ret]

    # 2. 準備垂直障礙
    if vertical_barrier_times is None:
        vb = pd.Series(close.index[-1], index=t_events)
    else:
        vb = vertical_barrier_times.reindex(t_events).fillna(close.index[-1])

    # 3. 準備 side
    if side_prediction is None:
        side = pd.Series(1.0, index=target.index)
    else:
        side = side_prediction.reindex(target.index).fillna(1.0)

    # 4. 建 events DataFrame
    events = pd.DataFrame({
        't1': vb,
        'trgt': target,
        'side': side
    }).dropna(subset=['trgt'])

    # 5. 並行計算 pt/sl 觸發時間
    df0 = mpPandasObj(func=applyPtSlOnT1,
                      pdObj={'molecule': events.index},
                      numThreads=num_threads,
                      close=close,
                      events=events,
                      pt_sl=pt_sl)

    # 6. 合併並取最早觸發時間
    df0 = df0.reindex(events.index)
    merged = pd.DataFrame({
        't1': events['t1'],
        'pt': df0['pt'],
        'sl': df0['sl']
    })
    events['t1'] = merged.min(axis=1)

    # 7. 若未提供 side_prediction，就不回傳 side 欄
    if side_prediction is None:
        events = events.drop(columns=['side'])

    return events


In [5]:
pt_sl = [1, 1]
min_ret = 0.003
triple_barrier_events = get_events(close=data['close'],
                                               t_events=cusum_events,
                                               pt_sl=pt_sl,
                                               target=vol,
                                               min_ret=min_ret,
                                               num_threads=4,
                                               vertical_barrier_times=vertical_barriers,
                                               side_prediction=None)

In [ ]:
import numpy as np
import pandas as pd

def get_indicator_matrix_np(events: pd.DataFrame, close_index: pd.DatetimeIndex) -> np.ndarray:
    """
    建立 indicator matrix（純 numpy 版）
      - 回傳 shape=(N_events, T_times) 的 0/1 陣列
      - events.index 和 events['t1'] 必須都能在 close_index 找到
    """
    # 1. 時間戳到 integer 位置的映射
    times = close_index.values  # array of np.datetime64
    pos_map = {t: i for i, t in enumerate(times)}
    
    starts = np.array([pos_map[t] for t in events.index.values], dtype=int)
    ends   = np.array([pos_map[t] for t in events['t1'].values], dtype=int)
    
    N = len(starts)
    T = len(times)
    ind = np.zeros((N, T), dtype=int)
    
    # 2. 把每個事件活躍區間 [start, end] 全部設成 1
    for i in range(N):
        ind[i, starts[i]: ends[i] + 1] = 1
    
    return ind

def average_uniqueness_np(indicator: np.ndarray) -> np.ndarray:
    """
    計算每個事件的 average uniqueness
      - indicator: shape (N, T)，0/1
      - 回傳 shape (N,) 的 weights
    """
    # 每個時間點 t 有幾個事件重疊
    overlap = indicator.sum(axis=0)             # (T,)
    mask    = overlap > 0                       # 只在 overlap>0 的位置計算
    # 瞬時唯一性矩陣 u[i,t]
    u = np.zeros_like(indicator, dtype=float)   # (N, T)
    u[:, mask] = indicator[:, mask] / overlap[mask]
    # 每個事件的持續長度
    durations = indicator.sum(axis=1)           # (N,)
    # 平均唯一性
    weights = u.sum(axis=1) / durations         # (N,)
    return weights

def sequential_bootstrap_np(indicator: np.ndarray, sample_size: int) -> np.ndarray:
    """
    Sequential Bootstrap（
      - indicator: shape (N, T)
      - sample_size: 欲抽出的事件數 (通常 = N)
      - 回傳 shape (sample_size,) 的 integer indices (0..N-1)
    """
    N = indicator.shape[0]
    weights = average_uniqueness_np(indicator)
    
    phi = np.empty(sample_size, dtype=int)
    remaining = np.arange(N)  # 剩下還沒抽的事件
    for k in range(sample_size):
        # 抽樣 pool
        pool = remaining
        if k == 0:
            probs = np.ones(pool.shape[0], dtype=float)
        else:
            probs = weights[pool]
        probs = probs / probs.sum()
        
        pick = np.random.choice(pool, p=probs)
        phi[k] = pick
        # 移除已選事件
        remaining = remaining[remaining != pick]
    
    return phi

def cal_weights(triple_barrier_events: pd.DataFrame, close_series: pd.Series) -> pd.DataFrame:
    """
    計算每個事件的權重
      - triple_barrier_events: DataFrame，包含欄位 ['t1','pt','sl']
      - close_series: 收盤價的 Series
      - 回傳 DataFrame，包含欄位 ['t1','pt','sl','weight']
    """
    # 1. 建 indicator matrix
    ind_mat = get_indicator_matrix_np(triple_barrier_events, close_series.index)
    
    # 2. 計算每個事件的 weight
    weights = average_uniqueness_np(ind_mat)
    
    # 3. 整理成跟原本一模一樣的 output DataFrame
    output = triple_barrier_events.copy()
    output['weight'] = weights
    
    return output



In [7]:
weights = cal_weights(triple_barrier_events, data['close'])

In [8]:
def get_bins(events, close):
    events = events.dropna(subset=['t1'])
    px = events.index.union(events['t1'].values).drop_duplicates()
    px = close.reindex(px, method='bfill')

    out = pd.DataFrame(index=events.index)
    out['ret'] = px.loc[events['t1'].values].values / px.loc[events.index] - 1

    if 'side' in events:
        out['ret'] *= events['side']
        out['bin'] = -1
        out.loc[out['ret'] > 0, 'bin'] = 1
    else:
        out['bin'] = np.sign(out['ret'])

    out = out[out['bin'] != 0]
    return out
labels  = get_bins(triple_barrier_events, data['close'])
labels['bin'].value_counts()

 1.0    2141
-1.0    2028
Name: bin, dtype: int64

## Make Features

In [9]:
import talib
import pandas as pd
from statsmodels.tsa.stattools import adfuller
# --- 以下 FFD helper functions 跟之前一樣 --- #
def get_ffd_weights(d: float, size: int, thresh: float = 1e-5) -> np.ndarray:
    w = [1.0]
    for k in range(1, size):
        w.append(w[-1] * ((-d + k - 1) / k))
    w = np.array(w)
    M = np.where(np.abs(w) > thresh)[0].max() + 1
    return w[:M]

def fractional_diff(series: pd.Series, d: float, thresh: float = 1e-5) -> pd.Series:
    x = series.values
    w = get_ffd_weights(d, len(x), thresh)
    if w.size == 1:
        return series.copy().rename(series.name)
    conv = np.convolve(x, w, mode='valid')
    idx = series.index[w.size-1:]
    return pd.Series(conv, index=idx, name=series.name)

def find_min_d(series: pd.Series, d_grid: np.ndarray) -> float:
    for d in d_grid:
        ffd = fractional_diff(series, d)
        pval = adfuller(ffd.dropna(), maxlag=1, regression='c')[1]
        if pval < 0.05:
            return d
    return d_grid[-1]

def compute_talib_features(data: pd.DataFrame,
                           periods: list = None,
                           apply_ffd: bool = True,
                           d_vals: np.ndarray = np.linspace(0, 1, 51),
                           ffd_thresh: float = 1e-5) -> pd.DataFrame:
    """
    接收含 open, high, low, close, volume 的 DataFrame，
    針對 periods 裡每個週期，計算一批 TA-Lib 指標，
    並回傳一個新的 DataFrame，裡面是所有這些技術指標特徵。
    """
    if periods is None:
        periods = [7, 14, 28, 50, 100]

    
    high, low, close, volume, log_ret = data['high'], data['low'], data['close'], data['volume'], np.log(data['close']).diff()
    # Log Returns
    features_df = pd.DataFrame(index=data.index)
    features_df['log_ret'] = log_ret
    for p in periods:
        # —— 波動率類 —— #
        features = dict({})
        features[f'atr_{p}']      = talib.ATR(high, low, close, timeperiod=p)
        upper, mid, lower         = talib.BBANDS(close, timeperiod=p, nbdevup=2, nbdevdn=2)
        features[f'bb_width_{p}']  = (upper - lower) / mid
        features[f'volatility_{p}'] = log_ret.rolling(window=p, min_periods=p, center=False).std()

        # —— 趨勢類 —— #
        features[f'sma_{p}']       = talib.SMA(close, timeperiod=p)
        features[f'ema_{p}']       = talib.EMA(close, timeperiod=p)
        features[f'adx_{p}']       = talib.ADX(high, low, close, timeperiod=p)
        features[f'plus_di_{p}']   = talib.PLUS_DI(high, low, close, timeperiod=p)
        features[f'minus_di_{p}']  = talib.MINUS_DI(high, low, close, timeperiod=p)
        features[f'dx_{p}']        = talib.DX(high, low, close, timeperiod=p)
        features[f'adxr_{p}']      = talib.ADXR(high, low, close, timeperiod=p)

        # —— 動量／均值回歸類 —— #
        features[f'rsi_{p}']       = talib.RSI(close, timeperiod=p)
        features[f'roc_{p}']       = talib.ROC(close, timeperiod=p)
        features[f'mom_{p}']       = talib.MOM(close, timeperiod=p)
        # features[f'autocorr_{p}'] = log_ret.rolling(window=100, min_periods=100, center=False).apply(lambda x: x.autocorr(lag=p), raw=False)
        # PPO = (EMA_fast - EMA_slow)/EMA_slow * 100
        fast = p
        slow = max(2*p, p+1)
        features[f'ppo_{p}'], features[f'ppo_signal_{p}'], features[f'ppo_hist_{p}'] = \
            talib.MACDEXT(close,
                          fastperiod=fast, fastmatype=0,
                          slowperiod=slow, slowmatype=0,
                          signalperiod=int(p/2), signalmatype=0)

        # KAMA
        features[f'kama_{p}']      = talib.KAMA(close, timeperiod=p)

        # Williams %R
        features[f'willr_{p}']     = talib.WILLR(high, low, close, timeperiod=p)

        # Stochastic
        slowk, slowd = talib.STOCH(
            high, low, close,
            fastk_period=p,
            slowk_period=max(3, p//3), slowk_matype=0,
            slowd_period=max(3, p//3), slowd_matype=0
        )
        features[f'stoch_k_{p}']   = slowk
        features[f'stoch_d_{p}']   = slowd
        features_df = pd.concat([features_df, pd.DataFrame(features)], axis=1)

    # —— 不需 timeperiod 的指標 —— #
    features_df['obv'] = talib.OBV(close, volume)
    features_df['adl'] = talib.AD(high, low, close, volume)
    features_df['sar'] = talib.SAR(high, low, acceleration=0.02, maximum=0.2)



    if apply_ffd:
        ffd_dict = {}
        for col in features_df.columns:
            # 1) 做微分前，自動檢定是否需要平穩化
            series = features_df[col].dropna()
            # 只對非平穩序列跑 FFD
            pval = adfuller(series, maxlag=1, regression='c')[1]
            if pval < 0.05:
                # 平穩就不動，直接填回原序列
                ffd_series = series
            else:
                # 非平穩就找 d* 並做分數階微分
                d_star     = find_min_d(series, d_vals)
                ffd_series = fractional_diff(series, d_star, thresh=ffd_thresh)
            # ffd_dict[f'{col}_ffd'] = ffd_series
            ffd_dict[f'{col}'] = ffd_series

        # 合併並對齊 index
        ffd_df = pd.DataFrame(ffd_dict)
        features_df = pd.concat([features_df, ffd_df], axis=1)
        features_df = features_df.dropna()
    return features_df

# 使用示範
feats = compute_talib_features(data,
                               periods=[7,14,28,50,100],
                               apply_ffd=True)
# print(feats.filter(like='_ffd').tail())
print(feats.tail())

                      log_ret     atr_7  bb_width_7  volatility_7  \
time                                                                
2024-12-31 15:33:00  0.000394  2.784474    0.005079      0.001177   
2024-12-31 15:38:00 -0.000857  2.795264    0.005262      0.001239   
2024-12-31 15:44:00 -0.000410  2.717369    0.004973      0.001151   
2024-12-31 15:51:00 -0.000031  2.574888    0.004409      0.001147   
2024-12-31 15:58:00  0.000341  2.681332    0.001876      0.001052   

                           sma_7        ema_7      adx_7  plus_di_7  \
time                                                                  
2024-12-31 15:33:00  2610.907143  2612.134702  28.737560  36.590239   
2024-12-31 15:38:00  2611.220000  2612.463526  28.121152  31.242004   
2024-12-31 15:44:00  2611.787143  2612.442645  26.466139  27.546491   
2024-12-31 15:51:00  2612.385714  2612.406984  23.499888  24.917807   
2024-12-31 15:58:00  2613.412857  2612.602738  23.076877  27.756102   

                  

In [10]:
idx = feats.index.intersection(labels.index)
feats = feats.loc[idx]
labels = labels.loc[idx]['bin']
weights =  weights.loc[idx]['weight']
t1 = triple_barrier_events.loc[idx]['t1']
print(feats.shape, labels.shape, weights.shape, t1.shape)

(4122, 208) (4122,) (4122,) (4122,)


## PCA

In [11]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

feats = feats.dropna()
class RollingPercentileTransformer(BaseEstimator, TransformerMixin):
    """
    對每一欄做滑動 percentile 計算，回傳每個時間點 t 
    欄位值在過去 window 期內的百分位 (0~1)。
    """
    def __init__(self, window: int = 252, min_periods: int = 1):
        self.window = window
        self.min_periods = min_periods

    def fit(self, X, y=None):
        # 不需要學任何東西
        return self

    def transform(self, X):
        # 假設 X 是 DataFrame
        X = pd.DataFrame(X).copy()
        for col in X.columns:
            # 每個 col 分別做 rolling.apply
            X[col] = (
                X[col]
                .rolling(window=self.window, min_periods=self.min_periods)
                .apply(lambda arr: (arr <= arr[-1]).sum() / len(arr), raw=True)
            )
        return X.values  # 回傳 numpy array 給後續 scaler

# === Pipeline 1: rolling percentile → z-score → PCA ===
pipe1 = Pipeline([
    ('roll_pct', RollingPercentileTransformer(window=252)),
    ('scaler',  StandardScaler()),
    ('pca',     PCA(n_components=0.95, whiten=False)),
])

# === Pipeline 2: z-score → PCA ===
pipe2 = Pipeline([
    ('scaler', StandardScaler()),
    ('pca',    PCA(n_components=0.95, whiten=False)),
])

# === 使用方式 ===
# 假設 feats 是一個 DataFrame，columns 就是你的所有技術指標
# e.g. feats = compute_talib_features(data)

# 1) 第一條流水線
X1 = pipe1.fit_transform(feats)  
# 2) 第二條流水線
X2 = pipe2.fit_transform(feats)


## MDA MDI SFI

#### Purged K Fold


In [ ]:
import numpy as np
import pandas as pd

class PurgedKFold:
    def __init__(self, n_splits=3, t1=None, pct_embargo=0.0):
        if not isinstance(t1, pd.Series):
            raise ValueError("t1 must be a pandas Series")
        self.n_splits = n_splits
        self.t1 = t1.sort_index()
        self.pct_embargo = pct_embargo

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

    def split(self, X, y=None, groups=None):
        if not X.index.equals(self.t1.index):
            raise ValueError("X and t1 must have the same index")
        n_samples = len(X)
        indices = np.arange(n_samples)
        # divide indices into contiguous chunks
        test_slices = np.array_split(indices, self.n_splits)
        mbrg = int(n_samples * self.pct_embargo)

        for slice_ in test_slices:
            i, j = slice_[0], slice_[-1] + 1
            test_idx = indices[i:j]

            # start‐time of test block
            t0 = self.t1.index[i]
            # end‐time of test block
            t1_max = self.t1.iloc[test_idx].max()
            # find the position just after t1_max
            max_t1_pos = self.t1.index.searchsorted(t1_max)

            # training before test block
            train_before = indices[self.t1.index < t0]
            # training after test + embargo
            train_after = indices[max_t1_pos + mbrg :]

            train_idx = np.concatenate([train_before, train_after])
            yield train_idx, test_idx


#### CVscore

In [ ]:
import numpy as np
from sklearn.base import clone
from sklearn.metrics import log_loss, accuracy_score

def cv_score(clf,
             X,
             y,
             sample_weight=None,
             scoring='neg_log_loss',
             t1=None,
             cv=3,
             pct_embargo=0.01):

    if scoring not in ['neg_log_loss', 'accuracy']:
        raise ValueError("scoring must be 'neg_log_loss' or 'accuracy'")

    pkf = PurgedKFold(n_splits=cv, t1=t1, pct_embargo=pct_embargo)
    scores = []

    for train_idx, test_idx in pkf.split(X):
        # 複製一份新的 model
        model = clone(clf)
        # fit
        if sample_weight is None:
            model.fit(X.iloc[train_idx], y.iloc[train_idx])
        else:
            model.fit(X.iloc[train_idx],
                      y.iloc[train_idx],
                      sample_weight=sample_weight.iloc[train_idx].values)
        # predict + score
        if scoring == 'neg_log_loss':
            prob = model.predict_proba(X.iloc[test_idx])
            sc = -log_loss(y.iloc[test_idx],
                           prob,
                           sample_weight=(None if sample_weight is None else sample_weight.iloc[test_idx].values),
                           labels=model.classes_)
        else:
            pred = model.predict(X.iloc[test_idx])
            sc = accuracy_score(y.iloc[test_idx],
                                pred,
                                sample_weight=(None if sample_weight is None else sample_weight.iloc[test_idx].values))
        scores.append(sc)
    return np.array(scores)




#### MDA MDI SFI 實作

In [29]:
import numpy as np
import pandas as pd
from sklearn.base import clone
from sklearn.metrics import log_loss, accuracy_score
from tqdm import tqdm

# 假設你已經有：
#   - PurgedKFold 實作  
#   - cv_score 函式  
# 並且都在 your_module 裡可以 import  


# 1) MDI Feature Importance
def feat_imp_mdi(fit, feat_names):
    """
    fit: 已訓練好的 tree‐ensemble（RandomForest, ExtraTrees…）
    feat_names: list of feature names
    return: pd.DataFrame with columns ['mean','std'] 純量化後的重要度
    """
    # 從每顆樹蒐集 feature_importances_
    df0 = pd.DataFrame(
        [tree.feature_importances_ for tree in fit.estimators_],
        columns=feat_names
    ).replace(0, np.nan)  # 如果 max_features=1，某些 tree 有 0
    imp = pd.concat({
        'median': df0.median(),
        'std' : df0.std() * df0.shape[0]**-0.5
    }, axis=1)
    # normalize to sum=1
    imp['median'] /= imp['median'].sum()
    imp.sort_values(by='median', ascending=False, inplace=True)
    return imp


# 2) MDA: 支援 X, y, sample_weight, t1 為 np.ndarray
def feat_imp_mda(clf,
                 X,
                 y,
                 sample_weight=None,
                 t1=None,
                 cv: int = 5,
                 pct_embargo: float = 0.01,
                 scoring: str = 'neg_log_loss'
                ) -> (pd.DataFrame, float):
    # --- 1) numpy → pandas ---
    if isinstance(X, np.ndarray):
        X = pd.DataFrame(X)
    if not isinstance(y, pd.Series):
        y = pd.Series(y, index=X.index)
    if sample_weight is not None and not isinstance(sample_weight, pd.Series):
        sample_weight = pd.Series(sample_weight, index=X.index)
    if t1 is not None and not isinstance(t1, pd.Series):
        t1 = pd.Series(t1, index=X.index)

    feat_names = list(X.columns)

    # --- 2) baseline score ---
    base_scores = cv_score(clf, X, y,
                           sample_weight=sample_weight,
                           scoring=scoring,
                           t1=t1,
                           cv=cv,
                           pct_embargo=pct_embargo)
    base_mean = base_scores.mean()

    # --- 3) 每個 feature permutation, 加進度條 ---
    diffs = []
    for col in tqdm(feat_names, desc="MDA permuting features"):
        Xp = X.copy()
        np.random.shuffle(Xp[col].values)
        perm_scores = cv_score(clf, Xp, y,
                               sample_weight=sample_weight,
                               scoring=scoring,
                               t1=t1,
                               cv=cv,
                               pct_embargo=pct_embargo)
        diffs.append(base_scores - perm_scores)

    diffs = np.vstack(diffs)
    imp_df = pd.DataFrame({
        'mean': diffs.mean(axis=1),
        'std' : diffs.std(axis=1) * diffs.shape[1]**-0.5
    }, index=feat_names)
    imp_df.sort_values(by='mean', ascending=False, inplace=True)
    return imp_df, base_mean

# 3) SFI: 支援 X, y, sample_weight, t1 為 np.ndarray
def SFI(feat_names: list,
                 clf,
                 X: pd.DataFrame,
                 y: pd.Series,
                 sample_weight=None,
                 t1=None,
                 cv: int = 5,
                 pct_embargo: float = 0.01,
                 scoring: str = 'neg_log_loss'
                ) -> pd.DataFrame:
    if isinstance(X, np.ndarray):
        X = pd.DataFrame(X, columns=feat_names)
    if not isinstance(y, pd.Series):
        y = pd.Series(y, index=X.index)
    if sample_weight is not None and not isinstance(sample_weight, pd.Series):
        sample_weight = pd.Series(sample_weight, index=X.index)
    if t1 is not None and not isinstance(t1, pd.Series):
        t1 = pd.Series(t1, index=X.index)

    imp = pd.DataFrame(columns=['mean', 'std'])
    for featName in feat_names:
        dfo = cv_score(clf, X=X[[featName]],  y = y,
                      sample_weight= sample_weight,
                      scoring=scoring, t1 = t1, cv = cv)
        imp.loc[featName, 'mean'] = dfo.mean()
        imp.loc[featName, 'std'] = dfo.std() * dfo.shape[0]**-0.5
        imp.sort_values(by='mean', ascending=False, inplace=True)
    return imp


### RF and compute MDI MDA SFI

In [16]:
col = [f"PCA_{i}" for i in range(X2.shape[1])]

In [17]:
# 用CV不用切割資料集
X = pd.DataFrame(X2, columns= col, index=feats.index)
y = labels.values
weights = weights.values
# t1 在上面定義好了

In [18]:
X

,PCA_0,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10,PCA_11,PCA_12,PCA_13,PCA_14,PCA_15,PCA_16,PCA_17,PCA_18,PCA_19
time,,,,,,,,,,,,,,,,,,,,
2021-01-22 16:00:00,-5.799092,-6.667734,2.831685,-4.376821,4.745149,4.974754,2.585366,-0.389366,4.162528,1.293853,1.544921,-1.215628,-2.283115,1.045526,-3.037757,1.378900,1.650350,-2.139362,-0.283402,-0.148872
2021-01-28 14:50:00,-12.438673,-5.509063,0.436969,-2.253595,4.779983,-4.238147,-5.008632,-0.920011,4.083533,0.551789,0.355830,0.725524,1.191631,0.931340,1.555752,0.790570,-0.931003,-1.237774,-2.571104,-0.070142
2021-01-28 15:58:00,-0.914997,-3.263566,-6.273350,5.583443,2.726280,2.424141,2.093870,1.179466,-0.341848,-0.887487,3.766684,0.869564,1.103646,1.653496,-1.123067,0.720345,0.562026,-1.178783,0.067104,-2.912669
2021-01-28 16:15:00,-4.921787,-4.013460,-4.221147,2.292808,3.155357,-1.735619,3.362364,-3.007337,-0.953520,1.139383,3.295104,-0.222116,1.022648,-0.279718,-0.456692,0.641452,-0.455475,-1.646982,-0.944321,-0.659289
2021-01-28 16:21:00,-8.490138,-4.423071,-2.284867,0.825572,4.501388,-2.716232,2.577636,-2.354038,0.865607,0.408851,2.383079,0.962312,1.365664,1.509007,-1.289653,0.346439,-0.605571,-1.815647,-1.195692,-0.297338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-20 16:32:00,-10.078947,10.976820,-2.200420,-5.326469,3.837847,0.166117,-1.960596,-0.426028,2.010870,-1.870346,0.778321,-1.961269,1.455997,1.611769,-0.650180,-2.265725,-0.386517,3.155148,0.285252,0.630826
2024-12-20 17:18:00,-11.826614,11.489787,-0.894275,-3.255975,6.131481,1.406690,-2.766024,-0.444787,1.300437,-1.161436,-0.928581,-2.642621,1.669947,1.000449,-0.558620,-2.406410,-0.332787,3.178175,0.084099,0.079467
2024-12-23 07:34:00,-6.772874,11.546762,-5.528706,-6.310889,3.496664,-0.622073,2.408875,1.316003,1.649202,0.434992,0.721245,-2.058231,2.431842,0.244633,-1.828676,-1.167709,0.122303,2.939340,0.339328,1.005489


In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

In [26]:
avgU = weights.mean()
clf = DecisionTreeClassifier(criterion='entropy', max_features='auto', class_weight='balanced')
clf = BaggingClassifier(estimator=clf, n_estimators=1000, max_samples=avgU)

In [21]:
# 1. MDI
clf_fit = clf.fit(X, y, sample_weight=weights)
mdi_imp = feat_imp_mdi(clf_fit, col)
print(mdi_imp)


c:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


          median       std
PCA_10  0.053839  0.000421
PCA_15  0.053119  0.000404
PCA_4   0.051702  0.000404
PCA_17  0.051656  0.000406
PCA_1   0.051380  0.000398
PCA_6   0.051320  0.000412
PCA_14  0.050161  0.000403
PCA_19  0.050110  0.000400
PCA_3   0.050083  0.000413
PCA_16  0.049945  0.000398
PCA_18  0.049815  0.000406
PCA_11  0.049576  0.000404
PCA_13  0.049454  0.000399
PCA_5   0.049263  0.000396
PCA_7   0.049117  0.000415
PCA_8   0.048678  0.000381
PCA_9   0.048476  0.000392
PCA_2   0.047525  0.000387
PCA_12  0.047445  0.000393
PCA_0   0.047337  0.000380


In [27]:
# 2. MDA
mda_imp, base = feat_imp_mda(
    clf, X, y, cv=5,
    sample_weight=weights,
    t1=t1, pct_embargo=0.01,
    scoring='neg_log_loss'
)
print(mda_imp, base)


MDA permuting features: 100%|██████████| 20/20 [37:45<00:00, 113.27s/it]

            mean       std
PCA_8   0.001491  0.000445
PCA_17  0.001370  0.000586
PCA_4   0.001166  0.000573
PCA_19  0.000805  0.000300
PCA_11  0.000736  0.000947
PCA_10  0.000730  0.001512
PCA_13  0.000630  0.000865
PCA_7   0.000580  0.000947
PCA_1   0.000416  0.001008
PCA_3   0.000381  0.000432
PCA_6   0.000376  0.000866
PCA_18  0.000369  0.000743
PCA_15  0.000005  0.001496
PCA_16 -0.000230  0.000801
PCA_5  -0.000251  0.000851
PCA_9  -0.000368  0.000534
PCA_12 -0.000480  0.000863
PCA_14 -0.000639  0.000500
PCA_2  -0.001096  0.001216
PCA_0  -0.001827  0.001479 -0.7009380955929598


In [30]:
# 3. SFI
sfi_imp = SFI(X.columns, clf, X, y, scoring='neg_log_loss', sample_weight=weights , cv=5, t1 = t1, pct_embargo=0.01)
print(sfi_imp)
sfi_imp.to_csv('sfi_imp.csv')

            mean         std
PCA_16 -0.836891  0.00624592
PCA_0    -0.8486   0.0191458
PCA_17 -0.851275   0.0107346
PCA_18 -0.851617   0.0136697
PCA_5  -0.853323  0.00954717
PCA_9  -0.857127   0.0134292
PCA_15 -0.861897   0.0043069
PCA_10 -0.863975   0.0094395
PCA_6  -0.865283   0.0111665
PCA_7  -0.865411   0.0114415
PCA_2  -0.866265  0.00833982
PCA_19 -0.866476  0.00921716
PCA_3  -0.872402   0.0115352
PCA_4  -0.875083  0.00595834
PCA_8  -0.875461  0.00624876
PCA_12 -0.882521  0.00575093
PCA_11  -0.88269   0.0120514
PCA_14  -0.88443   0.0143149
PCA_13 -0.897128  0.00950199
PCA_1  -0.909431   0.0287834
